# NNMD of zeolites using the pretrained models

In this notebook, we perform a NN-based MD NVE simulation using ASE and the [SchNet NN potential](https://github.com/learningmatter-mit/NeuralForceField). We will be using the second generation of zeolite models, as shown in [our paper](https://arxiv.org/abs/2101.11588).

The utilities at `nff` will be used to perform the MD simulation. `nglview` will be used to visualize the generated trajectories.

In [5]:
import os
import random
import numpy as np
from ase import Atoms, units
from ase.md.verlet import VelocityVerlet

from nff.io import NeuralFF, AtomsBatch, EnsembleNFF
from nff.md.nve import Dynamics
from nff.data import Dataset
import nff.utils.constants as const

import nglview as nv
from ase.io import Trajectory, read

## Loading the models and parameters

The dataset contains a PyTorch dataset with all the DFT data used to train the third generation of NN potentials. Here, we will use the pre-trained ensemble. For learning how to train the models using the SchNet architecture and the current dataset, check the tutorials at the original [NFF repo](https://github.com/learningmatter-mit/NeuralForceField).

In [2]:
dset = Dataset.from_file('../data/zeolite.pth.tar')

`DEVICE` sets the GPU used for evaluating the model. If you want to evaluate on a CPU, set `DEVICE = 'cpu'`. The models are stored at `/models/zeolite`, where `/` is the root folder of this repository.

In [3]:
DEVICE = 0

def get_ensemble_calc(device=DEVICE):
    PATH = '../models/zeolite'
    models = []
    for model_name in sorted(os.listdir(PATH)):
        m = NeuralFF.from_file(os.path.join(PATH, model_name), device=device).model
        models.append(m)

    return EnsembleNFF(models, device=device)

ensemble = get_ensemble_calc(device=DEVICE)

Here, we set the parameters for the MD simulation. For learning how to use these parameters within our code, check the tutorials at the original [NFF repo](https://github.com/learningmatter-mit/NeuralForceField).

In [33]:
def get_md_params(traj_filename, temperature=1000):
    return {
        'T_init': temperature,
        'time_step': 0.5,
        'thermostat': VelocityVerlet,  
        'thermostat_params': {'timestep': 0.5 * units.fs},
        'steps': 2 * 2000,
        'save_frequency': 40,
        'nbr_list_update_freq': 5,
        'thermo_filename': 'thermo.log',
        'traj_filename': traj_filename,
        'skip': 0
    }

Finally, we use the lowest energy conformation within the existing dataset (i.e. the ground state of ammonia) as a starting configuration for the MD simulation. `AtomsBatch` is a [wrapper within our NFF repo](https://github.com/learningmatter-mit/NeuralForceField/blob/master/nff/io/ase.py) and can be used to interface an ASE atom with NFF.

In [34]:
CUTOFF = 5.0

def get_md_atoms(dset=dset, cutoff=CUTOFF, device=DEVICE):
    props = random.choice(dset)

    atoms = AtomsBatch(
        positions=props['nxyz'][:, 1:],
        numbers=props['nxyz'][:, 0],
        cell=props['lattice'],
        pbc=True,
        cutoff=CUTOFF,
        props={'energy': 0, 'energy_grad': []},
        calculator=ensemble,
        device=device,
    )
    _ = atoms.update_nbr_list()
    
    return atoms

## Performing the MD simulation

Now, we perform the MD simulation using the parameters shown before.

In [35]:
TEMPERATURE = 1000

name = f'NVE_{TEMPERATURE}.traj'

atoms = get_md_atoms()
md_params = get_md_params(name, TEMPERATURE)
dyn = Dynamics(atoms, md_params)
traj = dyn.run()

Time[ps]      Etot[eV]     Epot[eV]     Ekin[eV]    T[K]
0.0000          21.1169       8.8837      12.2331   985.8



/home/dskoda/.conda/envs/htvs/lib/python3.7/site-packages/ase/atoms.py:921: VisibleDeprecationWarning: Use get_global_number_of_atoms() instead
  np.VisibleDeprecationWarning)


0.0200          20.7332      15.2592       5.4740   441.1

0.0400          20.5154      14.0231       6.4922   523.2

0.0600          20.6042      14.9808       5.6234   453.2

0.0800          20.4597      14.3222       6.1376   494.6

0.1000          20.5443      14.8327       5.7116   460.3

0.1200          20.2442      13.8381       6.4061   516.2

0.1400          20.3231      13.9752       6.3479   511.6

0.1600          20.0918      13.5825       6.5093   524.6

0.1800          20.1833      13.2984       6.8849   554.8

0.2000          20.2072      13.9056       6.3016   507.8

0.2200          20.2334      13.6448       6.5886   531.0

0.2400          20.2066      13.7613       6.4453   519.4

0.2600          20.1913      14.3460       5.8453   471.1

0.2800          20.2195      13.6512       6.5683   529.3

0.3000          20.2175      14.6771       5.5404   446.5

0.3200          20.1315      14.1899       5.9416   478.8

0.3400          19.9676      13.5944       6.3732   513.

## Visualizing the trajectory

Now we visualize the generated trajectory. The translation makes it easier to see the dynamics of the host within `nglview`.

In [36]:
filetraj = Trajectory(name)

newtraj = []
for at in filetraj:
    at.translate(at.cell.sum(0) * np.array([0.5, 0.5, 0.5]))
    at.wrap()
    newtraj.append(at)

In [37]:
view = nv.show_asetraj(newtraj)
view.add_unitcell()

view

NGLWidget(max_frame=100)